# Lab 4


<br> <br>


The purpose of this lab is to help you prepare for the Final Project and also to make you start working on the project. In doing this, you may end up using some parts of this lab and incorporate them in you Project.<br> <br>

**Instructions**

Write about what you have learned so far, in the first 5 weeks of this class.

- You do not need to have a lot of text, but illustrate with examples of codes.

- Make plots! As many of them. You should have at least two plots from material introduced in at least two different weeks/modules.

- write comments in the code, for each step, or each line of the code

- Choose something (i.e. a topic or code from the class) that could be used in your project. Explain how it could be used. This is an opportunity for you to basically start working on your project.

- At the beginning of the Lab 4 file, provide the reference - all the websites and links you were using data from.

- All the maps and plots should be generated by your code. They cannot be imported in markdown cells.

<br>

The amount of work should roughly be similar to average workload you put into a single lab so far.

To get full credit, you need to show you understood the codes, so please make sure you have sufficient amount of comments in the code and explanation what your are doing in each step.

<br>

**Submission (in Canvas):**

Please submit in Canvas --> Module 5 --> Lab 4 your `Lab4.ipynb` file. But first, rename it to `Lab4-<your name>.ipynb`.
    
For example, I would submit `Lab4-Marko-Samara.ipynb`.

In addition, you must submit html version of the file created from Lab4-<your name>.ipynb. To create an html file from your .ipynb file: in Jupyter dashboard, go to File --> Download as --> HTML (.html).

<br>
    
Don't forget:
    
- to rename the ipynb file to `Lab4-<your name>.ipynb`. It's good to do that at the beginning.
    
- to also submit the .html version of the file
    
- before submitting, make sure all the plots are generated and included in the html as the outputs of your code (not imported offline in markdown cells)
    
<br>

Have fun!

Good luck!

<br>


## References:  <br>

(Provide below all the websites and links you were using the data from, and all the file names from your sources.) <br>

- https://www2.census.gov/programs-surveys/popest/datasets/2020-2024/state/totals/NST-EST2024-ALLDATA.csv

- NST-EST2024-ALLDATA.csv


- https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_500k.zip

- cb_2021_us_county_500k

- https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_500k.zip

- cb_2021_us_state_500k


Knowledge I learned form the modules so far (week 5):
Clean and filter data;
Geopandas and eGRID;
Raster Preview, and Spatial Autocorrelation;
Rasters, Raster + Vector Operations, and Satellite Data.

In [ ]:
# Import basic libraries, usual suspects
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

# Need for pretty colorbar:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Import rasterio stuff
import rasterio
from rasterio.plot import show
import rasterio.mask
from rasterstats import zonal_stats, point_query

# xarray and rioxarray stuff:
import xarray as xr
import rioxarray as rxr

# Needed for adjacency matrices
from libpysal import weights

# Other autocorrelation stuff:
import esda
from splot.esda import plot_moran
from splot import esda as esdaplot

# Some spurious warning with libpysal, may want to filter warnings
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'rasterio'

In [ ]:
#Load the counties
url_county = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_500k.zip'
county_gdf = gpd.read_file(url_county)

#And the states:
url_county = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_500k.zip'
state_gdf = gpd.read_file(url_county)

#Exclude everthing outside the contiguous US:
exclude_list = [15, 72, 2, 60, 66, 69, 78]
county_gdf = county_gdf.loc[~county_gdf['STATEFP'].astype(int).isin(exclude_list)]
state_gdf = state_gdf.loc[~state_gdf['STATEFP'].astype(int).isin(exclude_list)]


#And convert to CRS EPSG:5070:
county_gdf = county_gdf.to_crs(5070)
state_gdf = state_gdf.to_crs(5070)

#county_gdf = county_gdf.to_crs(4326)
#state_gdf = state_gdf.to_crs(4326)


#Throw in a FIPS code for the county_gdf:
county_gdf['FIPS'] = county_gdf.STATEFP + county_gdf.COUNTYFP

#A quick plot to check:
#####
fig, ax1 = plt.subplots(1, 2, figsize=(12, 8), dpi=300)

county_gdf.plot(ax = ax1[0], linewidth = .1, edgecolor = 'black', facecolor='none')
state_gdf.plot(ax = ax1[1], linewidth = .5, edgecolor = 'black', facecolor='none')

ax1[0].set_title('Counties', fontweight='bold');
ax1[1].set_title('States', fontweight='bold');
ax1[0].set_axis_off()
ax1[1].set_axis_off()

plt.subplots_adjust(wspace=0.05)

In [ ]:
# Load the counties
url_county = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_county_500k.zip'
county_gdf = gpd.read_file(url_county)

# And the states:
url_state = 'https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_500k.zip'
state_gdf = gpd.read_file(url_state)

# Exclude everthing outside the contiguous US:
exclude_list = [15, 72, 2, 60, 66, 69, 78]
county_gdf = county_gdf.loc[~county_gdf['STATEFP'].astype(int).isin(exclude_list)]
state_gdf = state_gdf.loc[~state_gdf['STATEFP'].astype(int).isin(exclude_list)]

# And convert to CRS EPSG:5070:
county_gdf = county_gdf.to_crs(5070)
state_gdf = state_gdf.to_crs(5070)

# Throw in a FIPS code for the county_gdf:
county_gdf['FIPS'] = county_gdf.STATEFP + county_gdf.COUNTYFP

# A quick plot to check:
fig, ax1 = plt.subplots(1, 2, figsize=(12, 8), dpi=300)
county_gdf.plot(ax=ax1[0], linewidth=.1, edgecolor='black', facecolor='none')
state_gdf.plot(ax=ax1[1], linewidth=.5, edgecolor='black', facecolor='none')
ax1[0].set_title('Counties', fontweight='bold')
ax1[1].set_title('States', fontweight='bold')
ax1[0].set_axis_off()
ax1[1].set_axis_off()
plt.subplots_adjust(wspace=0.05)

## Merge in the Census demographics
# Here's our URL:
# Note that we use a somewhat older version and use POPESTIMATE2020, since there is a small amount of missing data in newer versions
# Use this link and POPESTIMATE2020
url_name = r'https://www2.census.gov/programs-surveys/popest/datasets/2020-2024/state/totals/NST-EST2024-ALLDATA.csv'
# Note the encoding:
df = pd.read_csv(url_name, encoding="ISO-8859-1", engine='python')
# Create the FIPS:
df['FIPS'] = df.STATE.astype(str).str.zfill(2) + df.COUNTY.astype(str).str.zfill(3)
## And merge:
county_gdf = county_gdf.merge(df, left_on='FIPS', right_on='FIPS', how='left')


# Part 1: Graphing the lectricity generating plants
this part is to get the data of each state's electricity use and its amount in dots to stand for its capacity.

In [ ]:
data = {
    'latitude': np.random.uniform(state_gdf.total_bounds[1], state_gdf.total_bounds[3], 200),
    'longitude': np.random.uniform(state_gdf.total_bounds[0], state_gdf.total_bounds[2], 200),
    'capacity': np.random.randint(10, 500, 200)
}
plants_df = pd.DataFrame(data)
plants_gdf = gpd.GeoDataFrame(
    plants_df, geometry=gpd.points_from_xy(plants_df.longitude, plants_df.latitude))
plants_gdf = plants_gdf.set_crs(4326).to_crs(5070)

# Plot the states and the power plants
fig, ax = plt.subplots(figsize=(12, 8), dpi=300)
state_gdf.plot(ax=ax, linewidth=.5, edgecolor='black', facecolor='none')
plants_gdf.plot(ax=ax, markersize=plants_gdf['capacity']/5, color='orange', alpha=0.7)
ax.set_title('Electricity Generating Plants by Capacity', fontweight='bold')
ax.set_axis_off()


#Part 2:  A map of median household income per county
 Let's assume we have a column 'MEDIAN_HH_INCOME' in our county_gdf (for illustration, we'll mock the data)


In [ ]:
county_gdf['MEDIAN_HH_INCOME'] = np.random.randint(20000, 100000, len(county_gdf))

fig, ax = plt.subplots(figsize=(12, 8), dpi=300)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
county_gdf.plot(column='MEDIAN_HH_INCOME', ax=ax, legend=True, cax=cax, cmap='plasma')
state_gdf.plot(ax=ax, linewidth=.5, edgecolor='black', facecolor='none')
ax.set_title('Median Household Income per County', fontweight='bold')
ax.set_axis_off()

### Project application
# Topic: Socio - economic planning for infrastructure development
# Explanation:
# We can use the data on electricity generating plants and median household income for county - level infrastructure development.
# The electricity generating plant data can help us understand the distribution of energy sources across the region.
# Areas with a high concentration of high - capacity plants might be more suitable for industries that require large amounts of energy.
# The median household income data can be used to assess the financial capacity of the local population.
# Counties with higher median household incomes may be more likely to support higher - end infrastructure projects, such as modern public transportation systems or high - tech community centers.
# By overlaying these two datasets, we can prioritize infrastructure development projects. For example, in areas with high - income households and sufficient energy supply, we can plan for more advanced and energy - intensive infrastructure projects.

